:header-kernel: python3



## Part 1, Forward Propagation



We start off by defining some hyperparameters, constants which establish the
structure and behavior of the network and are not updated as we train.

We're going to look at hours of sleep and hours of study, and predict our test score.

Here X is a 3x2 matrix, and Y is 3 x 1.

$3W_{11} + 5W_{21}$ per each neuron is what we want. We need to figure out
weights for each variable, for each example we have.

We can do this with matrix mulitplication (always remember [http://matrixmultiplication.xyz/](http://matrixmultiplication.xyz/)).

$$  \begin{bmatrix}
3 & 5 \\
5 & 1 \\
10 & 2
\end{bmatrix} \begin{bmatrix}
W_{11} & W_{12} & W_{13}\\
W_{21} & W_{22} & W_{33}\\
\end{bmatrix}
$$

$XW^{(1)} = Z^{(2)}$, where $Z^{(2)}$ is the activity of our second layer.

So each entry in Z is a sum of weighted inputs to each neuron. It has size 3x3:
one row for each example, one column for each hidden unit.

Next we will independently apply the activation function to each entry in Z.
We'll use the sigmoid function, leaning on NumPy, which rather conveniently
applies the function element-wise and returns the result with the same
dimensions it was given.

So we have $a^{(2)} = f(Z^{(2)})$. We'll then apply one more set of weights to
get our final output, and then run the activation function on that too.

$Z^{(3)} = a^{(2)} W^{(2)}$, and $\hat{y} = f(Z^{(3)})$



In [1]:
X = [[3, 5], 
     [5, 1], 
     [10, 2]]
y = [75, 82, 93]

[75, 82, 93]

In [1]:
import numpy as np
def sigmoid(z):
    return 1/(1 + np.exp(-z))

class Neural_Network(object):
    def __init__(self):
        self.inputLayersSize = 2
        self.outputLayersSize = 1
        self.hiddenLayersSize = 3

        self.W1 = np.random.randn(self.inputLayersSize, self.hiddenLayersSize)
        self.W2 = np.random.randn(self.hiddenLayersSize, self.outputLayersSize)

    def forward(self, X):
        Z2 = np.dot(X, self.W1)
        a2 = self.sigmoid(Z2)

        Z3 = np.dot(a2, self.W2)

        y_hat = self.sigmoid(Z3)

        return y_hat
        
    def sigmoid(self, z):
        return 1/(1 + np.exp(-z))

sigmoid(1), sigmoid(np.array([-1, 0, 1])), sigmoid(np.random.randn(3, 3))

NN = Neural_Network()
y_hat = NN.forward(X)
y_hat

array([[0.37793827],
       [0.48852971],
       [0.4885244 ]])